In [ ]:
# %pip install yfinance  # yFinance version 0.2.4 is required. If there are problems, reinstall yfinance. 'pip unistall yfinance'
# %pip install scipy
# %pip install matplotlib
# %pip install pyfolio
# %pip uninstall pyfolio
# %pip install git+https://github.com/quantopian/pyfolio
# %pip install panda

In [1]:
# Ignore printing all warnings
import warnings
warnings.filterwarnings('ignore')

# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pyfolio as pf
import datetime as dt
import pandas_datareader.data as web
import os
# import warnings

# print all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.stats import rankdata

In [ ]:
tickers_list = ['VNQ', 'EEM', 'DBC', 'SPY', 'TLT', 'SHY']
rebalance_unit = 'Month'
rebalance_freq = 1
rebalance_shift = 0
lb_period = 4
skipped_period = 0
no_played_ETFs = 3
sub_rank_weights = {'relMom' : 0.5, 'volatility' : 0.25, 'correlation' : 0.25}
abs_threshold = 0
start_date = '2006-01-01'
end_date = '2022-06-10'

In [3]:
tickers_list = ['SHY', 'IEF', 'TLT', 'TIP', 'LQD', 'HYG', 'BNDX', 'EMB', 'BIL']
# tickers_list = ['BNDX']
start_date = '2000-01-01'
end_date = '2023-01-05'
#adj_close_price = yf.download(tickers_list)['Adj Close']
adj_close_price = yf.download(tickers_list,start = pd.to_datetime(start_date) + pd.DateOffset(years= -2),end = pd.to_datetime(end_date) + pd.DateOffset(days= 1) )['Adj Close']
adj_close_price.to_csv('D:\Temp\pricestonovellbond.csv')

if len(adj_close_price['EMB']) > 0:
    print("Couldn't download EMB ticker data. You might need to reinstall yfinance python library. (ver 0.2.4 is required)")


[*********************100%***********************]  9 of 9 completed
Couldn't download EMB ticker data. You might need to reinstall yfinance python library. (ver 0.2.4 is required)


In [ ]:
tickers_list2 = ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'VNQ', 'DBC', 'GLD', 'HYG', 'LQD', 'TLT','IEF']
start_date = '2000-01-01'
end_date = '2023-12-31'
adj_close_price = yf.download(tickers_list2,start = pd.to_datetime(start_date) + pd.DateOffset(years= -2),end = pd.to_datetime(end_date) + pd.DateOffset(days= 1) )['Adj Close']
adj_close_price.to_csv('D:\Temp\kellerprotmom.csv')

In [ ]:
df = adj_close_price.copy()
df['Year'], df['Month'], df['Week'] = df.index.year, df.index.month, df.index.week
df['Monthly_Rb'] = df.Month != df.Month.shift(-1)
df['Weekly_Rb'] = df.Week != df.Week.shift(-1)
df['Daily_Rb'] = True
df['RebalanceUnit'] = df.Monthly_Rb if rebalance_unit == 'Month' else df.Weekly_Rb if rebalance_unit == 'Week' else df.Daily_Rb
df.RebalanceUnit = df.RebalanceUnit.shift(rebalance_shift).fillna(False) 
df.RebalanceUnit[0] = True
df['NoPeriod'] = df.RebalanceUnit.shift(1).cumsum()
df.NoPeriod[0] = 0
df['NoData'] = np.arange(len(df)) + 1
df['Rebalance'] = np.where((df.RebalanceUnit == True) & (df.NoPeriod.mod(rebalance_freq) == 0), True, False)
# offset_start_helper = df.groupby('NoPeriod')['NoData'].max()
# offset_start = offset_start_helper.shift(lb_period + skipped_period)
# df.merge(offset_start, on = 'NoPeriod')

In [ ]:
dailyret = adj_close_price/adj_close_price.shift(1) - 1

In [ ]:
lb_days_base = {'Month' : 21, 'Week' : 5, 'Day' : 1}
lb_days = lb_days_base[rebalance_unit] * lb_period
lb_days_with_skipped = lb_days_base[rebalance_unit] * (lb_period + skipped_period)

In [ ]:
rel_mom_rets = adj_close_price.shift(lb_days_base[rebalance_unit] * skipped_period) / adj_close_price.shift(lb_days_with_skipped) - 1

In [ ]:
sd_rets_helper = dailyret.rolling(lb_days).std()
sd_rets = sd_rets_helper.shift(lb_days_base[rebalance_unit] * skipped_period)
corr_rets_helper = dailyret.rolling(lb_days).corr()
corr_rets_helper2 = corr_rets_helper.groupby(corr_rets_helper.index.get_level_values('Date')).mean()
corr_rets = corr_rets_helper2.shift(lb_days_base[rebalance_unit] * skipped_period)

In [ ]:
rel_mom_rets_rank = rel_mom_rets.rank(axis = 1, ascending = False)
sd_rets_rank = sd_rets.rank(axis = 1, ascending = True)
corr_rets_rank = corr_rets.rank(axis = 1, ascending = True)
# total_rank_helper = sub_rank_weights['relMom'] * rel_mom_rets_rank + sub_rank_weights['volatility'] * sd_rets_rank + sub_rank_weights['correlation'] * corr_rets_rank - rel_mom_rets/100
total_rank_helper = sub_rank_weights['relMom'] * rel_mom_rets_rank + sub_rank_weights['volatility'] * sd_rets_rank + sub_rank_weights['correlation'] * corr_rets_rank
total_rank = total_rank_helper.rank(axis = 1, ascending = True)
no_selected_etfs = ((total_rank < no_played_ETFs + 1) & (rel_mom_rets > abs_threshold)).sum(1)
no_really_played_etfs = no_selected_etfs.where(no_selected_etfs > no_played_ETFs, no_played_ETFs)
etf_weights = (total_rank.divide(total_rank, axis = 0).divide(no_really_played_etfs, axis = 0)).where((total_rank < no_played_ETFs + 1) & (rel_mom_rets > abs_threshold), 0)
cash_weight = 1 - etf_weights.sum(axis = 1)

In [ ]:
def positions_pv(acp_p, rebalance_p, weights_p, cash_weight_p, start_date_p):
    adjclose = acp_p.fillna(1).to_numpy()
    rebalanceday = rebalance_p.to_numpy()
    weights = weights_p.fillna(0).to_numpy()
    cash_weight = cash_weight_p.to_numpy()
    no_rows, no_cols = acp_p.shape

    positions = np.zeros((no_rows, no_cols))
    pv = np.ones((no_rows,1))
    cash = np.ones((no_rows,1))
    start_ind = np.argmax(acp_p.index >= start_date_p)
    for i in range(start_ind, no_rows):
        pv[i] = 0
        for j in range(no_cols):
            if rebalanceday[i-1] == True:
                positions[i,j] = pv[i-1] * weights[i-1,j] / adjclose[i-1,j]                
            else:
                positions[i,j] = positions[i-1,j]
            pv[i] += positions[i,j] * adjclose[i,j]
        if rebalanceday[i-1] == True:
            cash[i] = pv[i-1] * cash_weight[i-1]
        else:
            cash[i] = cash[i-1]
        pv[i] += cash[i] 
    
    positions_df = pd.DataFrame(positions, index = acp_p.index, columns = acp_p.columns)
    positions_df_sel = positions_df.iloc[start_ind-1:no_rows,:]
    pv_df =  pd.DataFrame(pv, index = acp_p.index)
    pv_df_sel = pv_df.iloc[start_ind-1:no_rows,0]
    cash_df =  pd.DataFrame(cash, index = acp_p.index)
    cash_df_sel = cash_df.iloc[start_ind-1:no_rows,0]
    return positions_df_sel, cash_df_sel, pv_df_sel

In [ ]:
def positions_pv_ew(acp_p, rebalance_p, start_date_p):
    adjclose = acp_p.fillna(0).to_numpy()
    rebalanceday = rebalance_p.to_numpy()
    no_rows, no_cols = acp_p.shape
    no_ava_etfs = (acp_p > 0).sum(1)

    positions = np.zeros((no_rows, no_cols))
    pv = np.ones((no_rows,1))
    cash = np.ones((no_rows,1))
    start_ind = np.argmax(acp_p.index >= start_date_p)

    pv[start_ind] = 0
    for j in range(no_cols):
        positions[start_ind, j] = pv[start_ind - 1] * (1 / no_ava_etfs[start_ind - 1]) / adjclose[start_ind - 1, j] if adjclose[start_ind - 1, j] > 0 else 0
        pv[start_ind] += positions[start_ind, j] * adjclose[start_ind, j]
    
    for i in range(start_ind + 1, no_rows):
        pv[i] = 0
        for j in range(no_cols):
            if rebalanceday[i-1] == True:
                positions[i,j] = pv[i-1] * (1 / no_ava_etfs[i-1]) / adjclose[i-1,j] if adjclose[i-1, j] > 0 else 0                
            else:
                positions[i,j] = positions[i-1,j]
            pv[i] += positions[i,j] * adjclose[i,j]
        if rebalanceday[i-1] == True:
            cash[i] = 0
        else:
            cash[i] = 0
        pv[i] += cash[i] 
    
    positions_df = pd.DataFrame(positions, index = acp_p.index, columns = acp_p.columns)
    positions_df_sel = positions_df.iloc[start_ind-1:no_rows,:]
    pv_df =  pd.DataFrame(pv, index = acp_p.index)
    pv_df_sel = pv_df.iloc[start_ind-1:no_rows,0]
    cash_df =  pd.DataFrame(cash, index = acp_p.index)
    cash_df_sel = cash_df.iloc[start_ind-1:no_rows,0]
    return positions_df_sel, cash_df_sel, pv_df_sel

In [ ]:
pos, cash, pv = positions_pv(adj_close_price, df.Rebalance, etf_weights, cash_weight, start_date)
pos_ew, cash_ew, pv_ew = positions_pv_ew(adj_close_price, df.Rebalance, start_date)
strat_rets = pv / pv.shift(1) - 1
strat_rets_ew = pv_ew / pv_ew.shift(1) - 1
weights2 = etf_weights.copy()
weights2['cash'] = cash_weight
pf.create_full_tear_sheet(strat_rets, positions = weights2, benchmark_rets = strat_rets_ew)

In [ ]:
# # np.savetxt('outWeights.csv', pos, delimiter=';')
# with open('csv_data1.csv', 'w') as csv_file:
#     etf_weights.to_csv(path_or_buf=csv_file, sep = ';')